# Resume Screening with LLMs

**Goal**: Build a resume scoring system using structured outputs

## What You'll Learn
1. Load resume data from CSV
2. Use structured outputs to analyze resumes
3. Create a scoring system (0-100)

## Setup

In [ ]:
# Configuration
import json
from resume_utils import load_resumes, analyze_resume

OPENROUTER_API_KEY = ""  # Paste your key here

if not OPENROUTER_API_KEY or OPENROUTER_API_KEY.strip() == "":
    raise RuntimeError(
        "⚠️  Please set OPENROUTER_API_KEY above before running this notebook.\n"
        "Get your key from: https://openrouter.ai/keys"
    )

print("✓ API key configured")

## Part 1: Loading Resume Data

In [4]:
# Load all resumes into a dictionary
resumes = load_resumes('../data/resumes_final.csv')

print(f"Loaded {len(resumes)} resumes")
print(f"\nFirst 5 resume IDs: {list(resumes.keys())[:5]}")

Loaded 130 resumes

First 5 resume IDs: ['10089434', '10247517', '10265057', '10553553', '10641230']


In [5]:
# View a single resume
sample_id = list(resumes.keys())[0]
sample_resume = resumes[sample_id]

print(f"Resume ID: {sample_resume['ID']}")
print(f"\nResume text (first 500 characters):")
print("="*70)
print(sample_resume['Resume_str'][:500])
print("...")

Resume ID: 10089434

Resume text (first 500 characters):
         INFORMATION TECHNOLOGY TECHNICIAN I       Summary     Versatile Systems Administrator possessing superior troubleshooting skills for networking issues, end user problems, and network security. Experienced in server management, systems analysis, and offering in-depth understanding of IT infrastructure areas. Detail-oriented, independent, and focused on taking a systematic approach to solving complex problems. Demonstrated exceptional technical knowledge and skills while working with vari
...


## Part 2: Structured Output with LLMs

The `analyze_resume()` function takes:
1. **api_key**: Your OpenRouter API key
2. **prompt**: What you want to analyze
3. **resume_text**: The resume to analyze
4. **output_schema**: The JSON structure you want back
5. **model**: Which LLM to use (optional)

And returns structured JSON output.

## Example 1: Extract Technical Skills

In [6]:
# Define what we want to extract
prompt = "Extract the technical skills, programming languages, frameworks, and technologies from this resume."

# Define the output structure
output_schema = """
{
  "programming_languages": ["list of languages"],
  "frameworks_libraries": ["list of frameworks"],
  "databases": ["list of databases"],
  "cloud_platforms": ["list of cloud platforms"],
  "tools": ["list of development tools"]
}
"""

# Analyze the resume: note that this requires an output schema
result = analyze_resume(
    OPENROUTER_API_KEY,
    prompt,
    sample_resume['Resume_str'],
    output_schema
)

if result['error']:
    print(f"❌ Error: {result['error']}")
else:
    print("✓ Skills extracted successfully\n")
    print(json.dumps(result['result'], indent=2))
    print(f"\nTokens used: {result['usage'].get('total_tokens', 0)}")

✓ Skills extracted successfully

{
  "programming_languages": [
    "PowerShell",
    "VBScript",
    "HTML5",
    "CSS3"
  ],
  "frameworks_libraries": [
    ".Net Framework 4/4.5",
    "MVC 4"
  ],
  "databases": [],
  "cloud_platforms": [
    "Microsoft Azure",
    "Office 365"
  ],
  "tools": [
    "Active Directory",
    "Group Policy Objects",
    "Microsoft Exchange",
    "VMWare",
    "StorSimple",
    "Twinstrata",
    "Team Foundation Server",
    "Visual Studio",
    "Cacti",
    "Hyperion"
  ]
}

Tokens used: 925
